#### Packages

In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans, AgglomerativeClustering

#### Load Data

In [23]:
# Data Path
file109 = r'C:\Users\mayda\Python practice\TIER\database\109_TESG變數彙總表.xlsx'
file109_T = r'C:\Users\mayda\Python practice\TIER\database\109_TESG永續發展指標主表.xlsx'
file110 = r'C:\Users\mayda\Python practice\TIER\database\110_TESG變數彙總表.xlsx'
file110_T = r'C:\Users\mayda\Python practice\TIER\database\110_TESG永續發展指標主表.xlsx'
file111 = r'C:\Users\mayda\Python practice\TIER\database\111_TESG變數彙總表.xlsx'
file111_T = r'C:\Users\mayda\Python practice\TIER\database\111_TESG永續發展指標主表.xlsx'
fileINFO = "database/上市公司基本資料.xlsx"

dfINFO = pd.read_excel(fileINFO)
df_109 = pd.read_excel(file109)
df_109T = pd.read_excel(file109_T)
df_110 = pd.read_excel(file110)
df_110T = pd.read_excel(file110_T)
df_111 = pd.read_excel(file111)
df_111T = pd.read_excel(file111_T)

c:\Users\mayda\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


##### 產業代碼對應

In [24]:
# 創一個全新的df
desc_columns = ["代號", "名稱","TESG評等季度", "TESG評等公告日", "採用永續報告書年度", "採用永續報告書是否驗證", "SASB主產業", "SASB子產業"]

df = df_110.copy()
for col in df.columns:
    if col not in desc_columns:
        df = df.drop(columns=[col], axis=1) 

id_columns = ['代號', '主計處產業名']
dfINFO = dfINFO[id_columns]
df = df.merge(dfINFO, on='代號', how='outer')
df = df.dropna(subset=['主計處產業名'])

In [ ]:
# 創一個全新的df
desc_columns = ["代號", "名稱","TESG評等季度", "TESG評等公告日", "採用永續報告書年度", "採用永續報告書是否驗證", "SASB主產業", "SASB子產業"]

df = df_110.copy()
for col in df.columns:
    if col not in desc_columns:
        df = df.drop(columns=[col], axis=1) 

id_columns = ['代號', '主計處產業名']
dfINFO = dfINFO[id_columns]
df = df.merge(dfINFO, on='代號', how='outer')
df = df.dropna(subset=['主計處產業名'])

In [25]:
df.loc[df["主計處產業名"].str[0:2] == "08", "主計處產業名"] = 1
df.loc[df["主計處產業名"].str[0:2] == "09", "主計處產業名"] = 2
df.loc[df["主計處產業名"].str[0:2] == "10", "主計處產業名"] = 2
df.loc[df["主計處產業名"].str[0:2] == "11", "主計處產業名"] = 3
df.loc[df["主計處產業名"].str[0:2] == "12", "主計處產業名"] = 4
df.loc[df["主計處產業名"].str[0:2] == "13", "主計處產業名"] = 5
df.loc[df["主計處產業名"].str[0:2] == "14", "主計處產業名"] = 6
df.loc[df["主計處產業名"].str[0:2] == "15", "主計處產業名"] = 7
df.loc[df["主計處產業名"].str[0:2] == "16", "主計處產業名"] = 8
df.loc[df["主計處產業名"].str[0:2] == "17", "主計處產業名"] = 9
df.loc[df["主計處產業名"].str[0:2] == "18", "主計處產業名"] = 10
df.loc[df["主計處產業名"].str[0:2] == "19", "主計處產業名"] = 11
df.loc[df["主計處產業名"].str[0:2] == "20", "主計處產業名"] = 12
df.loc[df["主計處產業名"].str[0:2] == "21", "主計處產業名"] = 13
df.loc[df["主計處產業名"].str[0:2] == "22", "主計處產業名"] = 15
df.loc[df["主計處產業名"].str[0:2] == "23", "主計處產業名"] = 14
df.loc[df["主計處產業名"].str[0:2] == "24", "主計處產業名"] = 16
df.loc[df["主計處產業名"].str[0:2] == "25", "主計處產業名"] = 17
df.loc[df["主計處產業名"].str[0:2] == "26", "主計處產業名"] = 18
df.loc[df["主計處產業名"].str[0:2] == "27", "主計處產業名"] = 19
df.loc[df["主計處產業名"].str[0:2] == "28", "主計處產業名"] = 20
df.loc[df["主計處產業名"].str[0:2] == "29", "主計處產業名"] = 21
df.loc[df["主計處產業名"].str[0:2] == "30", "主計處產業名"] = 22
df.loc[df["主計處產業名"].str[0:2] == "31", "主計處產業名"] = 23
df.loc[df["主計處產業名"].str[0:2] == "32", "主計處產業名"] = 24
df.loc[df["主計處產業名"].str[0:2] == "33", "主計處產業名"] = 25
df.loc[df["主計處產業名"].str[0:2] == "34", "主計處產業名"] = 26

df.rename(columns={'主計處產業名': 'industry_id'}, inplace=True)

In [29]:
mask = df['industry_id'].str.len() >= 3
df = df[~mask]
print(df)

          代號   名稱 TESG評等季度   TESG評等公告日  採用永續報告書年度  採用永續報告書是否驗證  SASB主產業  \
14      1101   台泥  2021/12  2022/03/31     2020.0          1.0  提煉與礦產加工   
15      1102   亞泥  2021/12  2022/03/31     2020.0          1.0  提煉與礦產加工   
16      1103   嘉泥  2021/12  2022/03/31     2020.0          1.0  提煉與礦產加工   
17      1104   環泥  2021/12  2022/03/31     2020.0          0.0  提煉與礦產加工   
18      1108   幸福  2021/12  2022/03/31     2020.0          0.0  提煉與礦產加工   
...      ...  ...      ...         ...        ...          ...      ...   
2308    9105  NaN      NaN         NaN        NaN          NaN      NaN   
2310    9110  NaN      NaN         NaN        NaN          NaN      NaN   
2312  911622  NaN      NaN         NaN        NaN          NaN      NaN   
2313  911868  NaN      NaN         NaN        NaN          NaN      NaN   
2315    9136  NaN      NaN         NaN        NaN          NaN      NaN   

     SASB子產業 industry_id  
14      建築材料          14  
15      建築材料          14  
16      建築材料      

##### 計算年增率

In [30]:
def calculate_YoY(columns, df0, df1, df2, df):
    for column in columns:
        df[f"{column}_YoY"] = (df2[column]-df1[column]) / df1[column]
        df[f"{column}_base_YoY"] = (df1[column]-df0[column]) / df0[column]
    return df

columns_YoY = ["E1", "E7", "E9", "E10", "E15", "E16", "E17", "E20", "E21",
               "S1", "S2", "S4", "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14", "S15", "S17", "S19", "S20", "S21", "S24", 
               "G1", "G5", "G6", "G7", "G8", "G13", "G14", "G19", "G20", "G21", "G22", "G24"]
calculate_YoY(columns_YoY, df_109, df_110, df_111, df)

<ipython-input-30-a962aef25170>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{column}_YoY"] = (df2[column]-df1[column]) / df1[column]
<ipython-input-30-a962aef25170>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{column}_base_YoY"] = (df1[column]-df0[column]) / df0[column]


,代號,名稱,TESG評等季度,TESG評等公告日,採用永續報告書年度,採用永續報告書是否驗證,SASB主產業,SASB子產業,industry_id,E1_YoY,...,G19_YoY,G19_base_YoY,G20_YoY,G20_base_YoY,G21_YoY,G21_base_YoY,G22_YoY,G22_base_YoY,G24_YoY,G24_base_YoY
14,1101,台泥,2021/12,2022/03/31,2020.0,1.0,提煉與礦產加工,建築材料,14,0.346355,...,-0.333333,-0.571429,0.486486,inf,NaN,NaN,NaN,NaN,NaN,NaN
15,1102,亞泥,2021/12,2022/03/31,2020.0,1.0,提煉與礦產加工,建築材料,14,-0.964524,...,0.000000,-0.714286,-1.000000,inf,NaN,NaN,NaN,NaN,NaN,NaN
16,1103,嘉泥,2021/12,2022/03/31,2020.0,1.0,提煉與礦產加工,建築材料,14,1.399666,...,1.000000,-0.714286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,1104,環泥,2021/12,2022/03/31,2020.0,0.0,提煉與礦產加工,建築材料,14,44.836555,...,-0.714286,2.500000,0.981982,-0.333333,NaN,NaN,NaN,NaN,NaN,NaN
18,1108,幸福,2021/12,2022/03/31,2020.0,0.0,提煉與礦產加工,建築材料,14,-0.437077,...,0.666667,0.500000,-1.000000,-0.333333,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2308,9105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2310,9110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2312,911622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2313,911868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 計算增減

In [31]:
columns_YN = ["S22", "G9", "G10", "G11", "G12", "G15", "G16", "G17", "G18", "E19", "S26", "G23", "G25", "G26"]

def calculate_performance(col1, col2, col3, df0, df1, df):
    for col in df0.columns:
        if (col not in col1) & (col not in col2) & (col not in col3):
            df0.fillna(0)
            df1.fillna(0)          
            df[f"{col}_perf"] = df1[col]-df0[col]
    return df    
calculate_performance(columns_YoY, desc_columns, columns_YN, df_110, df_111, df)

<ipython-input-31-ce499b3f1fde>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_perf"] = df1[col]-df0[col]


,代號,名稱,TESG評等季度,TESG評等公告日,採用永續報告書年度,採用永續報告書是否驗證,SASB主產業,SASB子產業,industry_id,E1_YoY,...,S5_perf,S6_perf,S16_perf,S18_perf,S23_perf,S25_perf,G2_perf,G3_perf,G4_perf,E18_perf
14,1101,台泥,2021/12,2022/03/31,2020.0,1.0,提煉與礦產加工,建築材料,14,0.346355,...,0.0,0.0,-1.0,-1.0,2.0,-10.0,1.0,-1.0,-1.0,NaN
15,1102,亞泥,2021/12,2022/03/31,2020.0,1.0,提煉與礦產加工,建築材料,14,-0.964524,...,0.0,1.5,3.0,1.0,-3.0,-10.0,-1.0,0.0,0.0,NaN
16,1103,嘉泥,2021/12,2022/03/31,2020.0,1.0,提煉與礦產加工,建築材料,14,1.399666,...,1.0,-1.0,2.0,0.0,0.0,-1.0,1.0,0.0,0.0,NaN
17,1104,環泥,2021/12,2022/03/31,2020.0,0.0,提煉與礦產加工,建築材料,14,44.836555,...,0.5,0.0,0.0,-1.0,0.0,1.0,0.0,3.0,0.5,NaN
18,1108,幸福,2021/12,2022/03/31,2020.0,0.0,提煉與礦產加工,建築材料,14,-0.437077,...,-0.5,0.0,-2.0,0.0,2.0,-6.0,-0.5,-3.0,-0.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2308,9105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2310,9110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2312,911622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2313,911868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 合併最終成績

In [32]:
left = df
right = df_110T[["代號", "TESG等級", "TESG分數"]]
df = pd.merge(left, right, on="代號")
df.rename(columns = {'TESG等級': 'LEVEL_110'}, inplace=True)
df.rename(columns = {'TESG分數': 'TESG_110'}, inplace=True)

left = df
right = df_111T[["代號", "TESG等級", "TESG分數", "TESG升降等"]]
df = pd.merge(left, right, on="代號")
df.rename(columns = {'TESG等級': 'LEVEL_111'}, inplace=True)
df.rename(columns = {'TESG分數': 'TESG_111'}, inplace=True)
df.rename(columns = {'TESG升降等': 'TESG_change'}, inplace=True)

##### 輸出成csv

In [35]:
# 變成CSV檔
df.to_csv("database/ESG_TESG.csv", index=False, encoding='utf-8-sig')